<a href="https://colab.research.google.com/github/EllieZhangy/2023F_Advanced_ML/blob/main/Assignment_3_Yue_Zhang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3, Yue Zhang, Oct.20th

## Question 1
• Use papluca/language-identification dataset from Hugging Face to train a text classifier to identify the language of a text.

In [ ]:
pip install transformers datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

### 1.1 Load Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("papluca/language-identification")

In [ ]:
print(dataset)
print(dataset['train'][0])  # Print the first instance from the training set

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 70000
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 10000
    })
})
{'labels': 'pt', 'text': 'os chefes de defesa da estónia, letónia, lituânia, alemanha, itália, espanha e eslováquia assinarão o acordo para fornecer pessoal e financiamento para o centro.'}


In [ ]:
print(dataset['train'][:10])

{'labels': ['pt', 'bg', 'zh', 'th', 'ru', 'pl', 'ur', 'sw', 'tr', 'ur'], 'text': ['os chefes de defesa da estónia, letónia, lituânia, alemanha, itália, espanha e eslováquia assinarão o acordo para fornecer pessoal e financiamento para o centro.', 'размерът на хоризонталната мрежа може да бъде по реда на няколко километра ( km ) за на симулация до около 100 km за на симулация .', '很好，以前从不去评价，不知道浪费了多少积分，现在知道积分可以换钱，就要好好评价了，后来我就把这段话复制走了，既能赚积分，还省事，走到哪复制到哪，最重要的是，不用认真的评论了，不用想还差多少字，直接发出就可以了，推荐给大家！！', 'สำหรับ ของเก่า ที่ จริงจัง ลอง   honeychurch   ของเก่า ที่ ไม่   29   สำหรับ เฟอร์นิเจอร์ และ เงิน ไท ร้อง บริษัท ที่   122   สำหรับ ลาย คราม', 'Он увеличил давление .', 'S Jak sobie życzysz: Widzisz, jak Hitler zabija Żydów?', 'اس کے بارے میں ، سفید شادی کی شرح کے بعد سفید اورنمایاں طور پر سفید اورنمایاں طور پر .', 'Zabuni ya ushindani pia imekuwa rahisi kwa sifa ya kurudi kwenye mapendekezo yake ya grant , na dudovitz hivi karibuni ilikuwa kulea kwa ajili ya kuendeleza nyenzo za matangazo ya sl

### 1.2 Data Preprocessing

Create a Mapping for Language Codes:

In [ ]:
unique_langs = sorted(list(set(dataset['train']['labels'])))
lang2id = {lang: id for id, lang in enumerate(unique_langs)}
id2lang = {id: lang for lang, id in lang2id.items()}

num_labels = len(unique_langs)  # Determine the number of unique labels here

In [ ]:
print(id2lang)

{0: 'ar', 1: 'bg', 2: 'de', 3: 'el', 4: 'en', 5: 'es', 6: 'fr', 7: 'hi', 8: 'it', 9: 'ja', 10: 'nl', 11: 'pl', 12: 'pt', 13: 'ru', 14: 'sw', 15: 'th', 16: 'tr', 17: 'ur', 18: 'vi', 19: 'zh'}


Tokenize:

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def encode_batch(batch):
    batch['labels'] = [lang2id[label] for label in batch['labels']]
    encoding = tokenizer(batch['text'], truncation=True, padding='max_length', max_length=256, return_tensors='pt')
    return {**encoding, 'labels': batch['labels']}

dataset = dataset.map(encode_batch, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

### 1.3 Model Traning

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
   output_dir='./results',
   num_train_epochs=3,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   logging_dir='./logs',
   logging_steps=200,
   do_train=True,
   do_eval=True,
   evaluation_strategy="steps",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=dataset['train'],
   eval_dataset=dataset['validation'],
   compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).astype(float).mean().item()}
)

trainer.train()


Step,Training Loss,Validation Loss,Accuracy
200,0.678600,0.063967,0.989400
400,0.065300,0.063482,0.986400
600,0.068500,0.079757,0.983800
800,0.054200,0.052009,0.988600
1000,0.052200,0.036615,0.992000
1200,0.049000,0.049350,0.988700
1400,0.054100,0.038935,0.991800
1600,0.054800,0.064092,0.987300
1800,0.041700,0.068617,0.990900
2000,0.046900,0.060376,0.991000


Step,Training Loss,Validation Loss,Accuracy
200,0.678600,0.063967,0.989400
400,0.065300,0.063482,0.986400
600,0.068500,0.079757,0.983800
800,0.054200,0.052009,0.988600
1000,0.052200,0.036615,0.992000
1200,0.049000,0.049350,0.988700
1400,0.054100,0.038935,0.991800
1600,0.054800,0.064092,0.987300
1800,0.041700,0.068617,0.990900
2000,0.046900,0.060376,0.991000


TrainOutput(global_step=13125, training_loss=0.0315351917834509, metrics={'train_runtime': 2570.076, 'train_samples_per_second': 81.71, 'train_steps_per_second': 5.107, 'total_flos': 1.39135417344e+16, 'train_loss': 0.0315351917834509, 'epoch': 3.0})

### 1.4 Model Evaluation

Evaluate the model's performance on the test set:

In [ ]:
# Evaluate on the test set
results = trainer.evaluate(dataset['test'])
print("Test Accuracy with Average Embeddings:", results["eval_accuracy"])

Test Accuracy with Average Embeddings: 0.9931


Usage:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prediction function
def predict_language(text):
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=256, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Ensure inputs are on the correct device
    with torch.no_grad():
        logits = model(**inputs).logits
    pred_id = logits.argmax(-1).item()
    return id2lang[pred_id]

# Test the prediction function
test_text = "os chefes de defesa da estónia"
print(predict_language(test_text))  # Expected: 'pt'

pt


## Question 2

• Normally for text classification problems only [CLS] token is used to rep- resent the whole sentence. Try at least one more possibility on the above dataset: for example, use all the tokens, or use another token, or average the embeddings of all the non-padding tokens. Which performs better?

### 2.1 Average the embeddings of all the non-padding tokens：

In [ ]:
import torch
from torch.nn import functional as F
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers.models.distilbert.modeling_distilbert import DistilBertPreTrainedModel, DistilBertModel
from datasets import load_dataset

# Load dataset and preprocess
dataset = load_dataset("papluca/language-identification")

In [ ]:
unique_langs = sorted(list(set(dataset['train']['labels'])))
lang2id = {lang: id for id, lang in enumerate(unique_langs)}
id2lang = {id: lang for lang, id in lang2id.items()}

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
def encode_batch(batch):
    batch['labels'] = [lang2id[label] for label in batch['labels']]
    encoding = tokenizer(batch['text'], truncation=True, padding='max_length', max_length=256, return_tensors='pt')
    return {**encoding, 'labels': batch['labels']}

dataset = dataset.map(encode_batch, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Custom DistilBERT model using average embeddings
class DistilBertForAvgSequenceClassification(DistilBertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.distilbert = DistilBertModel(config)
        self.classifier = torch.nn.Linear(config.dim, config.num_labels)
        self.dropout = torch.nn.Dropout(config.seq_classif_dropout)
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)

        # Compute average embeddings
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_state.size()).float()
        sum_hidden_state = torch.sum(hidden_state * attention_mask_expanded, 1)  # sum across sequence length
        mean_hidden_state = sum_hidden_state / attention_mask_expanded.sum(1)  # mean pooling

        pooled_output = self.dropout(mean_hidden_state)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else logits


In [ ]:
# Train the model
model = DistilBertForAvgSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(unique_langs))

training_args = TrainingArguments(
   output_dir='./results_avg_embedding',
   num_train_epochs=3,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   logging_dir='./logs_avg_embedding',
   logging_steps=200,
   do_train=True,
   do_eval=True,
   evaluation_strategy="steps",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=dataset['train'],
   eval_dataset=dataset['validation'],
   compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).astype(float).mean().item()}  # Include a metric for evaluation
)

trainer.train()



Some weights of DistilBertForAvgSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
200,0.397100,0.068299,0.984000
400,0.058600,0.060942,0.987300
600,0.068400,0.061755,0.988100
800,0.047800,0.066838,0.985700
1000,0.055200,0.055706,0.988000
1200,0.042600,0.064041,0.989000
1400,0.063000,0.055541,0.989600
1600,0.050100,0.060548,0.988600
1800,0.048600,0.084425,0.986800
2000,0.036500,0.061293,0.991000


TrainOutput(global_step=13125, training_loss=0.026842183039301918, metrics={'train_runtime': 2558.0797, 'train_samples_per_second': 82.093, 'train_steps_per_second': 5.131, 'total_flos': 1.372304037888e+16, 'train_loss': 0.026842183039301918, 'epoch': 3.0})

In [ ]:
# Evaluate on the test set
results = trainer.evaluate(dataset['test'])
print("Test Accuracy with Average Embeddings:", results["eval_accuracy"])

Test Accuracy with Average Embeddings: 0.9904


### 2.2 Use another token - Concatenating Tokens




In [ ]:
import torch
from torch.nn import functional as F
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers.models.distilbert.modeling_distilbert import DistilBertPreTrainedModel, DistilBertModel
from datasets import load_dataset

dataset = load_dataset("papluca/language-identification")

In [ ]:
unique_langs = sorted(list(set(dataset['train']['labels'])))
lang2id = {lang: id for id, lang in enumerate(unique_langs)}
id2lang = {id: lang for lang, id in lang2id.items()}

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
def encode_batch(batch):
    batch['labels'] = [lang2id[label] for label in batch['labels']]
    encoding = tokenizer(batch['text'], truncation=True, padding='max_length', max_length=256, return_tensors='pt')
    return {**encoding, 'labels': batch['labels']}

dataset = dataset.map(encode_batch, batched=True)

In [ ]:
class DistilBertForConcatSequenceClassification(DistilBertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.distilbert = DistilBertModel(config)
        self.concat_size = 4  # number of tokens to concatenate
        self.classifier = torch.nn.Linear(config.dim * self.concat_size, config.num_labels)
        self.dropout = torch.nn.Dropout(config.seq_classif_dropout)
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)

        concat_embedding = hidden_state[:, :self.concat_size].reshape(hidden_state.size(0), -1)  # Taking the first few embeddings and concatenating

        pooled_output = self.dropout(concat_embedding)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else logits


In [ ]:
# Train the model
model = DistilBertForAvgSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(unique_langs))

training_args = TrainingArguments(
   output_dir='./results_con_embedding',
   num_train_epochs=3,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   logging_dir='./logs_con_embedding',
   logging_steps=200,
   do_train=True,
   do_eval=True,
   evaluation_strategy="steps",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=dataset['train'],
   eval_dataset=dataset['validation'],
   compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).astype(float).mean().item()}  # Include a metric for evaluation
)

trainer.train()


Some weights of DistilBertForAvgSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
200,0.350500,0.049768,0.986700
400,0.065100,0.042901,0.988800
600,0.076100,0.110587,0.976400
800,0.051600,0.074622,0.986100
1000,0.045400,0.068516,0.988900
1200,0.042100,0.058273,0.989400
1400,0.054200,0.052043,0.989000
1600,0.054500,0.164913,0.964300
1800,0.053200,0.129136,0.972500
2000,0.037900,0.108386,0.978800


TrainOutput(global_step=13125, training_loss=0.025561480399540492, metrics={'train_runtime': 2563.966, 'train_samples_per_second': 81.904, 'train_steps_per_second': 5.119, 'total_flos': 1.372304037888e+16, 'train_loss': 0.025561480399540492, 'epoch': 3.0})

In [ ]:
# Evaluate on the test set
results = trainer.evaluate(dataset['test'])
print("Test Accuracy with Average Embeddings:", results["eval_accuracy"])

Test Accuracy with Average Embeddings: 0.991


[CLS] Token:
- Training Accuracy: 99.4%
- Test Accuracy: 99.3%

Averaging Tokens:
- Training Accuracy: 99.12%
- Test Accuracy: 99.04%

Concatenating Tokens:
- Training Accuracy: 99.2%
- Test Accuracy: 99.1%

The [CLS] token method shows the best performance with the highest test accuracy. It's closely followed by the Concatenating Tokens method, and then the Averaging All Tokens method.

All three methods yield highly impressive results with accuracies above 99% for both training and test datasets. This speaks to the robustness of the training steps employed, as well as the efficacy of the model architectures used.


